# Course Assignment Problem (a small exploration)

We have 500 course exams and 3000 students, each course is ensured to have at least one student enrolled, besides, we have 70 time slot. We want to assign each course exam to a time slot, each time slot is given a cost in advance, each student that have courses exammed at the slot will have corresponding cost (This cost indicates students' welfare, because students will feel angry for assignments such as "Not willing to have exams at 8:00 A.M., or Not willing to have exams during weekends, or Not willing to have exams at the end of exams weeks").

We hope to assign each course exam to a time slot such that the total cost is minimized. 

CONSTRS:

1. Students cannot have two exams simultaneously at one slot;
2. Each course exam must be assigned to a time slot;

CAUTION:

1. Different students have different number of courses, range from 3 to 10.

In [39]:
# data generate ... 

import random
import numpy as np
import gurobipy as grb


def generate_intervals_weights():
    weights = np.array([[1] * 7 for _ in range(10)])
    weights[0] += 3 # 8:00
    weights[5] += 3 # 8:00
    weights[4] += 1 # night exam
    weights[9] += 1 # night exam 
    weights[0:5, -2: ] += 2 # weekend 
    weights[5:10, -2: ] += 4 # weekend
    weights[9, -2: ] += 3 # "goal keeper"
    # The result is just like:
    # [[4 4 4 4 4 6 6]
    #  [1 1 1 1 1 3 3]
    #  [1 1 1 1 1 3 3]
    #  [1 1 1 1 1 3 3]
    #  [2 2 2 2 2 4 4]
    #  [4 4 4 4 4 8 8]
    #  [1 1 1 1 1 5 5]
    #  [1 1 1 1 1 5 5]
    #  [1 1 1 1 1 5 5]
    #  [2 2 2 2 2 8 8]]
    # Two weeks in total, each day has 5 time slots: 8:00~10:00, 10:30~12:30, 14:00~16:00, 16:30~18:30, 19:00~21:00
    # Apparently, students don't like 8:00~10:00, and 19:00~21:00 exams.
    # Apparently too, students don't like exams during weekends.
    weights = weights.flatten()
    return list(weights)

def generate_student_course_relation(num_courses, num_students):
    # 存储学生参加的课程
    student_courses = {}
    course_students = {}
    # 确保每个课程都有学生参加
    for course in range(num_courses):
        if course not in course_students:
            course_students[course] = []
        student = random.randint(0, num_students - 1)
        if student in student_courses:
            student_courses[student].append(course)
        else:
            student_courses[student] = [course]
        course_students[course].append(student)
    
    # 生成每个学生参加的课程数在 2~10 之间
    for student in range(num_students):
        if student not in student_courses:
            student_courses[student] = []
        while len(student_courses[student]) < random.randint(2, 10):
            course = random.randint(0, num_courses - 1)
            if course not in student_courses[student] and len(student_courses[student]) < 10:
                student_courses[student].append(course)
                course_students[course].append(student)
    return student_courses, course_students

def check_feasibility(result, student_courses, course_students, weights, num_courses, num_students, intervals):
    feasible = True
    # check if the result is okay ... 
    # 1. if all courses are assigned ... 
    if len(result.keys()) != num_courses:
        feasible = False
    # 2. if students time conflict ... 
    if not feasible:
        return feasible    
    for student in range(num_students):
        check_interv = set()
        for course in student_courses[student]:
            if result[course] in check_interv:
                print(f"Wrong! Student {student} has conflict with course {course} at interval {result[course]}")
                feasible = False 
            else:
                check_interv.add(result[course])
    return feasible


def solver(student_courses, course_students, weights, num_courses, num_students, intervals):
    model = grb.Model("CourseSchedule")
    x = dict()
    for course in range(num_courses):
        for interv in range(intervals):
            x[course, interv] = model.addVar(
                vtype=grb.GRB.BINARY, name= f"{course}_{interv}"
            )
            # if allocate course i to interval j, 0-1 binary

    # set objective func
    model.modelSense = grb.GRB.MINIMIZE
    model.setObjective(grb.quicksum(
        x[course, interv] * weights[interv] * len(course_students[course]) for course in range(num_courses) for interv in range(intervals)
    ))
    # set constraints ... 
    for course in range(num_courses):
        model.addConstr(grb.quicksum(x[course, interv] for interv in range(intervals)) == 1, f"C_{course}_fill")
    for student in range(num_students):
        for interv in range(intervals):
            model.addConstr(grb.quicksum(x[course, interv] for course in student_courses[student]) <= 1, f"C_{student}_{interv}_leq1")
    
    model.optimize()
    result = dict()
    if model.status == grb.GRB.status.OPTIMAL:
        ofv = model.getObjective().getValue()
        print(f"Objective Value: {ofv}")
        for course in range(num_courses):
            for interv in range(intervals):
                if x[course, interv].x > 0.5:
                    result[course] = interv
                    print(f"Course {course}, with {len(course_students[course])} students, is allocated to interval {interv}")
    return result    

    
if __name__ == "__main__":
    num_courses = 500
    num_students = 3000
    student_courses, course_students = generate_student_course_relation(num_courses, num_students)
    weights = generate_intervals_weights()
    intervals = 70
    result = solver(student_courses, course_students, weights, num_courses, num_students, intervals)
    check = check_feasibility(result, student_courses, course_students, weights, num_courses, num_students, intervals)
    print(f"Pass feasibility check? {check}")
    

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M3 Pro
Thread count: 11 physical cores, 11 logical processors, using up to 11 threads

Optimize a model with 210500 rows, 35000 columns and 962850 nonzeros
Model fingerprint: 0xd9098d38
Variable types: 0 continuous, 35000 integer (35000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 36862.000000
Presolve removed 7000 rows and 0 columns
Presolve time: 1.70s
Presolved: 203500 rows, 35000 columns, 994490 nonzeros
Variable types: 0 continuous, 35000 integer (35000 binary)
Found heuristic solution: objective 20565.000000

Deterministic concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...


Use crossover to convert LP symmetric solution to basic solution...
Concurrent spin time: 0.16s

Solved wit

In [ ]:
# checked for feasibility.